In [1]:
import os

# Set Java 17 for PySpark
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = '/opt/homebrew/opt/openjdk@17/bin:' + os.environ.get('PATH', '')


from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.appName("Spark SQL Query Dataframes").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/17 20:38:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data_path = '/Users/zlatavorobeva/Documents/GitHub/SQL Int/SQL_Intermediate/Spark/Data'

In [3]:
json_df_path = data_path + "/utilization.json"
df_util = spark.read.format("json").load(json_df_path)

In [4]:
df_util.show(10)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

In [5]:
df_util.createOrReplaceTempView("utilization")

In [6]:
csv_df_path = data_path + "/server_name.csv"
df_server = spark.read.csv(csv_df_path,header=True)

In [7]:
df_server.show()

+---------+-----------+
|server_id|server_name|
+---------+-----------+
|      100| 100 Server|
|      101| 101 Server|
|      102| 102 Server|
|      103| 103 Server|
|      104| 104 Server|
|      105| 105 Server|
|      106| 106 Server|
|      107| 107 Server|
|      108| 108 Server|
|      109| 109 Server|
|      110| 110 Server|
|      111| 111 Server|
|      112| 112 Server|
|      113| 113 Server|
|      114| 114 Server|
|      115| 115 Server|
|      116| 116 Server|
|      117| 117 Server|
|      118| 118 Server|
|      119| 119 Server|
+---------+-----------+
only showing top 20 rows


In [8]:
df_server.createOrReplaceTempView("server_name")

In [9]:
df_count = spark.sql("SELECT DISTINCT server_id FROM utilization ORDER BY server_id")
df_count.show()

+---------+
|server_id|
+---------+
|      100|
|      101|
|      102|
|      103|
|      104|
|      105|
|      106|
|      107|
|      108|
|      109|
|      110|
|      111|
|      112|
|      113|
|      114|
|      115|
|      116|
|      117|
|      118|
|      119|
+---------+
only showing top 20 rows


In [10]:
spark.sql("SELECT min(server_id), max(server_id) FROM utilization").show()

+--------------+--------------+
|min(server_id)|max(server_id)|
+--------------+--------------+
|           100|           149|
+--------------+--------------+



In [11]:
spark.sql("SELECT * FROM server_name").show()

+---------+-----------+
|server_id|server_name|
+---------+-----------+
|      100| 100 Server|
|      101| 101 Server|
|      102| 102 Server|
|      103| 103 Server|
|      104| 104 Server|
|      105| 105 Server|
|      106| 106 Server|
|      107| 107 Server|
|      108| 108 Server|
|      109| 109 Server|
|      110| 110 Server|
|      111| 111 Server|
|      112| 112 Server|
|      113| 113 Server|
|      114| 114 Server|
|      115| 115 Server|
|      116| 116 Server|
|      117| 117 Server|
|      118| 118 Server|
|      119| 119 Server|
+---------+-----------+
only showing top 20 rows


In [12]:
df_join = spark.sql("SELECT u.server_id, sn.server_name, u.session_count \
                    FROM utilization u \
                    INNER JOIN server_name sn \
                    ON u.server_id = sn.server_id  ").show()

+---------+-----------+-------------+
|server_id|server_name|session_count|
+---------+-----------+-------------+
|      100| 100 Server|           47|
|      100| 100 Server|           43|
|      100| 100 Server|           62|
|      100| 100 Server|           50|
|      100| 100 Server|           43|
|      100| 100 Server|           48|
|      100| 100 Server|           58|
|      100| 100 Server|           58|
|      100| 100 Server|           62|
|      100| 100 Server|           45|
|      100| 100 Server|           47|
|      100| 100 Server|           60|
|      100| 100 Server|           57|
|      100| 100 Server|           44|
|      100| 100 Server|           47|
|      100| 100 Server|           66|
|      100| 100 Server|           65|
|      100| 100 Server|           66|
|      100| 100 Server|           42|
|      100| 100 Server|           63|
+---------+-----------+-------------+
only showing top 20 rows


In [13]:
from pyspark.sql.types import Row

In [17]:
sc = spark.sparkContext
df_dup = sc.parallelize([Row(server_name = '101 Server', cpu_utilization=85, session_count=100), 
                         Row(server_name = '101 Server', cpu_utilization=80, session_count=90), 
                         Row(server_name = '102 Server', cpu_utilization=85, session_count=80), 
                         Row(server_name = '102 Server', cpu_utilization=85, session_count=80)]).toDF()

In [18]:
df_dup.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|          100|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [19]:
df_dup.drop_duplicates().show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|          100|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [21]:
df_dup.dropDuplicates().show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|          100|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [ ]:
df_dup.drop_duplicates(['server_name']).show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|          100|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



25/12/18 00:46:55 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 994437 ms exceeds timeout 120000 ms
25/12/18 00:46:55 WARN SparkContext: Killing executors is not supported by current scheduler.
25/12/18 00:46:56 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$